In [69]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Starting up
# -----------
db = sqlite3.connect('L21DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

# Exercise
# --------
# Inner join: join by candidate id, display contrib (last, first) and cand. lastname
query = ''' SELECT contributors.last_name, contributors.first_name,
            candidates.last_name
            FROM contributors INNER JOIN candidates  
            ON contributors.candidate_id = candidates.id '''
display(viz_tables(['contributors.last_name',
                    'contributors.first_name',
                    'candidates.last_name'], query))

# same, but for specified candidate last name
query = ''' SELECT contributors.last_name, contributors.first_name,
            candidates.last_name
            FROM contributors INNER JOIN candidates  
            ON contributors.candidate_id = candidates.id 
            WHERE candidates.last_name="Paul"'''
display(viz_tables(['contributors.last_name',
                    'contributors.first_name',
                    'candidates.last_name'], query))

# Exercise
# --------
# Average contributions grouped by candidate name
#query = """SELECT A.nA, A.attr, nB, B.attr FROM B LEFT JOIN A ON A.idA = B.idB"""
query = ''' SELECT contributors.last_name, contributors.first_name,
            candidates.last_name
            FROM contributors INNER JOIN candidates  
            ON contributors.candidate_id = candidates.id 
            WHERE candidates.last_name="Paul"'''

,contributors.last_name,contributors.first_name,candidates.last_name
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
5,Akin,Mike,Huckabee
6,Akin,Rebecca,Huckabee
7,Aldridge,Brittni,Huckabee
8,Allen,John D.,Huckabee
9,Allen,John D.,Huckabee


,contributors.last_name,contributors.first_name,candidates.last_name
0,Schuff,Bryan,Paul
1,Hobbs,James,Paul
2,Ranganath,Anoop,Paul
3,Nystrom,Michael,Paul
4,Muse,Nina,Paul
5,Waddell,James,Paul
6,Brucks,William,Paul
7,Kuehn,David,Paul
8,Verster,Jeanette,Paul
9,Uihlein,Richard,Paul


# Table A B Check

In [70]:
db = sqlite3.connect('ab.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS a")
cursor.execute("DROP TABLE IF EXISTS b")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE a (
               nA TEXT, 
               attr INT, 
               idA INT)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE b (
               nB TEXT, 
               attr INT, 
               idB INT)''')

db.commit() # Commit changes to the database

with open('./a.txt', 'r') as f:
    data = f.read().split('\n')
    
for row in data[1:-1]:
    n, attr, id_num = row.split('\t')
    # insert data into table
    cursor.execute('''INSERT INTO a
               (nA, attr, idA)
               VALUES (?, ?, ?)''', 
                (n, attr, id_num))
    db.commit()
    
with open('./b.txt', 'r') as f:
    data = f.read().split('\n')
    
for row in data[1:-1]:
    n, attr, id_num = row.split('\t')
    # insert data into table
    cursor.execute('''INSERT INTO b
               (nB, attr, idB)
               VALUES (?, ?, ?)''', 
                (n, attr, id_num))
    db.commit()

display(viz_tables(['nA', 'attr', 'idA'], 'SELECT * FROM a'))
display(viz_tables(['nB', 'attr', 'idB'], 'SELECT * FROM b'))

#query = """SELECT A.nA, A.attr, nB, B.attr FROM A LEFT JOIN B ON B.idB = A.idA"""
query = """SELECT A.nA, A.attr, nB, B.attr FROM B LEFT JOIN A ON A.idA = B.idB"""

display(viz_tables(['nA', 'A.attr', 'nB', 'B.attr'], query))

,nA,attr,idA
0,s1,23,0
1,s2,7,2
2,s3,15,2
3,s4,31,0


,nB,attr,idB
0,t1,60,0
1,t2,14,7
2,t3,22,2


,nA,A.attr,nB,B.attr
0,s1,23.0,t1,60
1,s4,31.0,t1,60
2,None,NaN,t2,14
3,s2,7.0,t3,22
4,s3,15.0,t3,22
